In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
!pip install fastapi uvicorn[standard] streamlit pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 81.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.2/452.2 kB 33.9 MB/s eta 0:00:00


In [1]:
import torch
from transformers import AutoTokenizer
from unsloth import FastLanguageModel

# Path to the fine-tuned model on Hugging Face Hub
model_path = "abdulsamad99/CONSTITUTION_USA-fine-tuning"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load model (Unsloth returns a tuple of model and tokenizer)
model, _ = FastLanguageModel.from_pretrained(
    model_name=model_path,
    load_in_4bit=True,  # Keep 4-bit quantization for efficiency
    dtype=torch.float16,
)

/tmp/ipython-input-1186196466.py:3: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

chat_template.jinja:   0%|          | 0.00/462 [00:00<?, ?B/s]

==((====))==  Unsloth 2025.8.10: Fast Llama patching. Transformers: 4.55.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/131M [00:00<?, ?B/s]

Unsloth 2025.8.10 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [2]:
import torch
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import threading
from transformers import AutoTokenizer, AutoModelForCausalLM
from pyngrok import ngrok, conf

# Set ngrok auth token for FastAPI
FASTAPI_NGROK_TOKEN = "enter your ngrok auth token foe fastapi"  # Replace with your actual token
conf.get_default().auth_token = FASTAPI_NGROK_TOKEN



# FastAPI app setup
app = FastAPI(title="U.S. Constitution Q&A API")

class QuestionRequest(BaseModel):
    question: str
    max_new_tokens: int = 150

def ask_model(question, max_new_tokens=150):
    prompt = f"Question: {question}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        temperature=0.7,
        top_p=0.9,
    )
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Answer:" in answer:
        answer = answer.split("Answer:")[1].strip()
        for marker in ["##", "###", "[problem]:", "[solution]:", "**Problem:**"]:
            if marker in answer:
                answer = answer.split(marker)[0].strip()
    return answer

@app.post("/generate")
async def generate(request: QuestionRequest):
    response = ask_model(request.question, request.max_new_tokens)
    return {"question": request.question, "answer": response}

def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Start FastAPI in a background thread
threading.Thread(target=run_fastapi, daemon=True).start()
print("✅ FastAPI running locally at http://localhost:8000")

# Create ngrok tunnel for FastAPI
ngrok_tunnel_fastapi = ngrok.connect(8000, bind_tls=True)
print(f"🌐 FastAPI ngrok URL: {ngrok_tunnel_fastapi.public_url}")

✅ FastAPI running locally at http://localhost:8000


INFO:     Started server process [6360]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🌐 FastAPI ngrok URL: https://74a4334187ba.ngrok-free.app


In [3]:
# Write app.py (Streamlit UI)
%%writefile app.py
import streamlit as st
import requests
import time

FASTAPI_URL = "enter your generated FastAPI ngrok URL"  # Update if using FastAPI's ngrok URL
st.set_page_config(
    page_title="US Constitution Q&A Bot",
    page_icon="📜",
    layout="centered"
)
st.title("US Constitution Q&A Bot")

if "messages" not in st.session_state:
    st.session_state.messages = []

for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

if prompt := st.chat_input("Ask me anything about the US Constitution..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        try:
            payload = {"question": prompt, "max_new_tokens": 150}
            response = requests.post(f"{FASTAPI_URL}/generate", json=payload)
            if response.status_code == 200:
                answer = response.json().get("answer", "")
                for chunk in answer.split():
                    full_response += chunk + " "
                    message_placeholder.markdown(full_response + "▌")
                    time.sleep(0.05)
                message_placeholder.markdown(full_response)
            else:
                message_placeholder.error("❌ Error from FastAPI.")
        except Exception as e:
            message_placeholder.error(f"⚠️ Could not connect: {e}")
    st.session_state.messages.append({"role": "assistant", "content": full_response})



Overwriting app.py


In [4]:
# Run Streamlit with a separate ngrok tunnel
from pyngrok import ngrok, conf

# Set ngrok auth token for Streamlit
STREAMLIT_NGROK_TOKEN = "enter your another ngrok auth token for streamlit ui"  # Replace with your actual token
conf.get_default().auth_token = STREAMLIT_NGROK_TOKEN

# Create ngrok tunnel for Streamlit
ngrok_tunnel_streamlit = ngrok.connect(8501, bind_tls=True)
print(f"🌐 Streamlit ngrok URL: {ngrok_tunnel_streamlit.public_url}")

🌐 Streamlit ngrok URL: https://1bbd1bca2305.ngrok-free.app


In [ ]:
# Run Streamlit
!streamlit run app.py --server.port 8501 --server.headless true




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.151.156:8501

INFO:     34.16.151.156:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     34.16.151.156:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     34.16.151.156:0 - "POST /generate HTTP/1.1" 200 OK
INFO:     34.16.151.156:0 - "POST /generate HTTP/1.1" 200 OK
